In [2]:
using ITensors
using Distributions

In [10]:

N = 10
ttotal = 10000
szcent=zeros(ttotal)
timesa=[]
s = siteinds("S=1/2", N; conserve_qns=false)
t=1
e=0.03
for ks in 1:10
  state = ["Up" ,"Up" ,"Up" ,"Up" ,"Up","Dn","Dn","Dn","Dn","Dn" ] # Initialize psi to be a product state (up state)
  global psi
  psi = productMPS(s,state)  
  cutoff = 1E-9
  tau = 1
  dj=0.1*2*pi   
  sss=rand(Uniform(0, 2*pi), N)
  jall=rand(Uniform(2*pi-dj, 2*pi+dj), N-1) 

  # Make gates (1,2),(2,3),(3,4),...
  gates = ITensor[]
  szop=[]
  for j in 1:N-1 
    s1 = s[j]
    s2 = s[j + 1]
    hj =                                  # two-qubit Hamiltonian used to evolve the MPS
     jall[j]*op("Sz", s1) * op("Sz", s2) 
    Gj = exp(-im * tau * hj)
    push!(gates, Gj)
  end  
  for k in 1:N
    sdr=s[k]
    hj1=op("Sx", sdr)
    Gjdriving = exp(-im *(pi-e)* hj1)
    push!(gates, Gjdriving)
  end
  aa=0
  for t in 0:tau:ttotal
    Sz = expect(psi, ["Sz"])  #Calculate expectation values (magnetization)
    #println(Sz[1][5])
    global psi
    push!(szop,((-1)^aa)*Sz[1][5]) 
    #println(szop)
    psi = apply(gates, psi; cutoff=cutoff) #Apply the gates for the next time step
    normalize!(psi) 
    aa+=1
  end  
  for i in 1:ttotal
   szcent[i]+=szop[i]                    #Average magnetization due to disorder realizations
  end
end  
println(szcent/10)
ENV["GKS_ENCODING"]="utf-8"
using Plots
using LaTeXStrings                       
plot(1:ttotal, szcent/10,title=L"$<Sz>_{middle}$", label=["|↑↑↑↑↑↓↓↓↓↓>"], linewidth=3) #Plot magnetization
plot!(xscale=:log10, minorgrid=true)